In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import win32com.client as win32
import subprocess
from pathlib import Path
import base64
import json
from bs4 import BeautifulSoup
import pandas_gbq as pbq

pd.set_option('display.max_rows', None)

### prelim_transactions 
- Duration: 3 min 14 sec  
- Bytes processed: 1.41 TB    

### combined_transactions
- Duration: 8 sec
- Bytes processed: 99.76 GB 


In [13]:
base_query = """
SELECT 
  b.master_artist,
  b.r2_project_description,
  a.sub_account_name as account,
  a.product_type,
  a.usage_type,
  a.usage_group,
  a.transaction_date,
  a.units,
  a.album_adjusted_units
FROM `umg-finance.consumption.prelim_transactions` as a
JOIN (
  SELECT 
    product_id,
    release_album, 
    project_release_date, 
    master_artist, 
    r2_project_description
  FROM `umg-finance.metadata.microstrategy_product` WHERE r2_project_number = '{r2_project_number}' 
) as b
ON a.product_id = b.product_id
WHERE transaction_date BETWEEN '{start_date}' AND '{end_date}'
"""

# query_roar_prelim = base_query.format(table='prelim_transactions')
query_roar = base_query.format(
  r2_project_number='007377515',
  start_date = '2020-04-03',
  end_date = '2020-10-03'
  )
query_burn = base_query.format(
  r2_project_number='007426222',
  start_date = '2022-04-08',
  end_date = '2022-08-03'
  )


def pull_data(q):
    df = pbq.read_gbq(q, project_id='umg-us')
    return df



In [14]:
# df_prelim = pull_data(query_roar_prelim)
# df_prelim.to_pickle(r'C:\Users\KamilcS\OneDrive - Universal Music Group\projects\ADHOC_ALBUMS_COMPARE\data\passion_roar_6month_preliminary_transactions.pkl')

# df_prelim = pd.read_pickle('./data/passion_roar_6month_preliminary_transactions.pkl')
# df_combined = pull_data(query_roar_combined)

df_roar = pull_data(query_roar)
df_burn = pull_data(query_burn)

df_roar.to_pickle(r'C:\Users\KamilcS\OneDrive - Universal Music Group\projects\ADHOC_ALBUMS_COMPARE\data\passion_roar_6month_prelim_transactions.pkl')
df_burn.to_pickle(r'C:\Users\KamilcS\OneDrive - Universal Music Group\projects\ADHOC_ALBUMS_COMPARE\data\passion_burn_6month_prelim_transactions.pkl')

# df_roar = pd.read_pickle(r'C:\Users\KamilcS\OneDrive - Universal Music Group\projects\ADHOC_ALBUMS_COMPARE\data\passion_roar_6month_combined_transactions.pkl')
# df_burn = pd.read_pickle(r'C:\Users\KamilcS\OneDrive - Universal Music Group\projects\ADHOC_ALBUMS_COMPARE\data\passion_burn_6month_combined_transactions.pkl')

Downloading: 100%|██████████| 1472105/1472105 [01:27<00:00, 16828.59rows/s]


In [10]:
df_prelim.head()

,master_artist,r2_project_description,account,transaction_date,units
0,Passion,Passion/Roar (Live)/LP10,google,2020-04-05,1
1,Passion,Passion/Roar (Live)/LP10,google,2020-04-05,3
2,Passion,Passion/Roar (Live)/LP10,rhapsody,2020-04-05,2
3,Passion,Passion/Roar (Live)/LP10,google,2020-04-05,1
4,Passion,Passion/Roar (Live)/LP10,rhapsody,2020-04-05,1


In [8]:
def prepare_data(df):

    df['account'] = df['account'].str.lower()
    df['raw_streams'] = np.where(
        df['usage_group'].str.contains('Streaming', regex=False), df['units'], 0
    )
    df['master_album'] = df['r2_project_description'].apply(lambda x: x.split('/')[-1])

    df = df.groupby(['transaction_date', 'master_album']).agg({
        'units': sum,
        'album_adjusted_units' : sum,
        'raw_streams': sum,

    }).sort_index().reset_index()

    df.index = df.index + 1

    df.index.name = 'days_post_release'

    # df = df.reset_index()

    return df

In [ ]:
df_prelim.head()

In [9]:
df_roar = prepare_data(df_prelim)

df_burn = prepare_data(pd.read_pickle(r'C:\Users\KamilcS\OneDrive - Universal Music Group\projects\ADHOC_ALBUMS_COMPARE\data\passion_burn_6month_combined_transactions.pkl'))


KeyError: 'usage_group'

In [ ]:
joined = pd.concat([df_roar, df_burn]).reset_index()
joined.head(50)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 9), dpi=150)

raw_streams_roar = joined.query('master_album == "LP10"')['Raw Streams'].sum()
raw_streams_burn = joined.query('master_album == "LP12"')['Raw Streams'].sum()

fig.suptitle('Passion Raw Streams by LP', size = 40, weight="bold", y=1.03, x=0.52)
sns.lineplot(
    data=joined, 
    # x='Weeks Post Release', 
    x = 'Days Post Release',
    y='Raw Streams', 
    hue='Master Album', 
    ax=ax,
    linewidth = 2,
    
).set(
    # xlabel='Weeks Post Release',
    xlabel='Days Post Release',
    ylabel='Raw Stream Units [k]',
    title=f'Roar       : {raw_streams_roar:,.0f}\nBurn Bright: {raw_streams_burn:,.0f}'
    # ylabel='Raw Streams'
    )


ylabels = [
    '{:,.0f}'.format(x) for x in ax.get_yticks() 
    /1000
    ]

sns.despine(left=True)
ax.set_yticklabels(ylabels)

In [ ]:
df_burn.head()

In [ ]:
print(df_roar.groupby('account')['units'].sum().sum())
print(df_burn.groupby('account')['units'].sum().sum())

In [ ]:
print(df_roar.usage_group.unique())
print(df_burn.usage_group.unique())

In [ ]:
print(df_roar.product_type.unique())
print(df_burn.product_type.unique())

In [ ]:
raw_streams_roar = df_roar.query('usage_group == "(LP10) Roar"')['Raw Streams'].sum()
raw_streams_burn = df_burn.query('usage_group == "(LP12) Burn Bright"')['Raw Streams'].sum()

In [ ]:
df_roar.master_artist.unique()

In [ ]:
df_burn.master_artist.unique()